<h4>Thesis</h4>
<h6>These models try predict if the score of the home team(principal) is larger than the visitor team, utilizing KNN. <br>The variables X are: "home_odds", "draw_odds", "away_odds", "PSH", "PSD", "PSA"<br>
The variables Y are: "lay_home"</h6>

<h6>lay (LAY) = draw and loss of home team</h6>

<h6>When the quantity of the score of the visitor team is larger than the home team(principal) then assign 0 to the field "lay_home". When the quantity of the score of the visitor team is minus than of the home team(principal) then assign 1 to field "lay_home".</h6>

<h6>*The secret to increase the hit rate is to choose good variables. The better the variables the higher the hit rate will be.*</h6>

<h5>Import dataset</h5>
<h6>Importing the dataset from file .csv.</h6>

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings

pd.set_option("display.min_rows", 200)
pd.set_option("display.max_rows", 600)
pd.set_option("display.max_columns", 100)

# Premier League - England
premier_league_2016_2017 = pd.read_csv("https://www.football-data.co.uk/mmz4281/1617/E0.csv")
premier_league_2017_2018 = pd.read_csv("https://www.football-data.co.uk/mmz4281/1718/E0.csv")
premier_league_2018_2019 = pd.read_csv("https://www.football-data.co.uk/mmz4281/1819/E0.csv")
premier_league_2019_2020 = pd.read_csv("https://www.football-data.co.uk/mmz4281/1920/E0.csv")
premier_league_2020_2021 = pd.read_csv("https://www.football-data.co.uk/mmz4281/2021/E0.csv")
premier_league_2021_2022 = pd.read_csv("https://www.football-data.co.uk/mmz4281/2122/E0.csv")
premier_league_2022_2023 = pd.read_csv("https://www.football-data.co.uk/mmz4281/2223/E0.csv")

df = pd.concat([premier_league_2016_2017, premier_league_2017_2018, premier_league_2018_2019, premier_league_2019_2020, premier_league_2020_2021, premier_league_2021_2022, premier_league_2022_2023], axis=0, ignore_index=True)
#df = pd.concat([premier_league_2021_2022, premier_league_2022_2023], axis=0, ignore_index=True)
df["League"] = "Premier League"
df = df[["League", "Date", "HomeTeam", "AwayTeam", "B365H", "B365D", "B365A", "PSH", "PSD", "PSA", "FTHG", "FTAG"]]
df.columns = ["league", "date", "home", "away", "home_odds", "draw_odds", "away_odds", "PSH", "PSD", "PSA", "home_goals_ft", "away_goals_ft"]
df = df.astype({"date":"datetime64"})
df.dropna(inplace=True)
df = df.convert_dtypes(infer_objects=True)
df.reset_index(inplace=True, drop=True)
df.tail()

,league,date,home,away,home_odds,draw_odds,away_odds,PSH,PSD,PSA,home_goals_ft,away_goals_ft
2506,Premier League,2023-02-18,Nott'm Forest,Man City,8.50,5.50,1.30,10.30,5.67,1.33,1,1
2507,Premier League,2023-02-18,Wolves,Bournemouth,1.67,3.60,6.00,1.69,3.86,5.78,0,1
2508,Premier League,2023-02-18,Newcastle,Liverpool,2.50,3.50,2.70,2.57,3.58,2.82,0,2
2509,Premier League,2023-02-19,Man United,Leicester,1.62,4.33,5.25,1.65,4.34,5.37,3,0
2510,Premier League,2023-02-19,Tottenham,West Ham,1.75,3.60,4.75,1.87,3.76,4.47,2,0


<h5>Feature Engineering and Preprocessing</h5>
<h6>When the quantity of the score of the home team(principal) is larger than the visitor team then assign 0 to the field "lay_home". When the quantity of the score of the visitor team is minus than of the home team(principal) then assign 1 to field "lay_home".</h6>

In [9]:
df.loc[(df["home_goals_ft"] > df["away_goals_ft"]), "lay_home"] = 0
df.loc[(df["home_goals_ft"] <= df["away_goals_ft"]), "lay_home"] = 1
df.tail()

,league,date,home,away,home_odds,draw_odds,away_odds,PSH,PSD,PSA,home_goals_ft,away_goals_ft,lay_home
2506,Premier League,2023-02-18,Nott'm Forest,Man City,8.50,5.50,1.30,10.30,5.67,1.33,1,1,1.0
2507,Premier League,2023-02-18,Wolves,Bournemouth,1.67,3.60,6.00,1.69,3.86,5.78,0,1,1.0
2508,Premier League,2023-02-18,Newcastle,Liverpool,2.50,3.50,2.70,2.57,3.58,2.82,0,2,1.0
2509,Premier League,2023-02-19,Man United,Leicester,1.62,4.33,5.25,1.65,4.34,5.37,3,0,0.0
2510,Premier League,2023-02-19,Tottenham,West Ham,1.75,3.60,4.75,1.87,3.76,4.47,2,0,0.0


<h5>Defining X and Y</h5>

In [10]:
X = df.loc[:, ["home_odds", "draw_odds", "away_odds", "PSH", "PSD", "PSA"]]
y = df.loc[:, "lay_home"]

<h5>Pre processing (Standard Scaler)</h5>

In [11]:
std = StandardScaler()
X = std.fit_transform(X)

<h5>Split Train and Test</h5>

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(X)

(1883, 6) (628, 6) (1883,) (628,)
[[-0.26958056 -0.66173725 -0.36323789 -0.26070791 -0.67878625 -0.3887904 ]
 [-0.4338339  -0.66173725 -0.09386176 -0.42908599 -0.69838516 -0.15748291]
 [ 0.05892613 -0.59581504 -0.54641366  0.05962113 -0.60692358 -0.55664185]
 ...
 [-0.22851722 -0.52989282 -0.48176339 -0.21964008 -0.50892903 -0.46452825]
 [-0.58987458  0.01726159  0.06776392 -0.59746407 -0.01242331  0.05744882]
 [-0.53649224 -0.4639706  -0.03998653 -0.50711486 -0.39133557 -0.12677838]]


In [13]:
knn = KNeighborsClassifier(n_neighbors=3, metric="minkowski", p=2)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

<h5>Evaluate the Model</h5>

In [16]:
y_predictions = knn.predict(X_test)

print(f"Accuracy score (training): {knn.score(X_train, y_train)}")
print(f"Accuracy score (validation): {knn.score(X_test, y_test)}\n")
print(f"Confusion Matrix:\n {confusion_matrix(y_test, y_predictions)}\n")
print(f"Classification Report:\n {classification_report(y_test, y_predictions)}")
print(f"Accuracy score:\n {accuracy_score(y_test, y_predictions)}")

Accuracy score (training): 0.7902283590015932
Accuracy score (validation): 0.6257961783439491

Confusion Matrix:
 [[163 110]
 [125 230]]

Classification Report:
               precision    recall  f1-score   support

         0.0       0.57      0.60      0.58       273
         1.0       0.68      0.65      0.66       355

    accuracy                           0.63       628
   macro avg       0.62      0.62      0.62       628
weighted avg       0.63      0.63      0.63       628

Accuracy score:
 0.6257961783439491


<h5>Informations about model</h5>

In [17]:
print(f"Games that was predicted as been 0 and was 0: {confusion_matrix(y_test, y_predictions)[0][0]}")
print(f"Games that was predicted as been 0 and was 1: {confusion_matrix(y_test, y_predictions)[1][0]}")
print(f"Games that was predicted as been 1 and was 0: {confusion_matrix(y_test, y_predictions)[0][1]}")
print(f"Games that was predicted as been 1 and was 1: {confusion_matrix(y_test, y_predictions)[1][1]}")

test_total_matches = confusion_matrix(y_test, y_predictions)[0][0] + \
                     confusion_matrix(y_test, y_predictions)[0][1] + \
                     confusion_matrix(y_test, y_predictions)[1][0] + \
                     confusion_matrix(y_test, y_predictions)[1][1]

entry_games = confusion_matrix(y_test, y_predictions)[0][1] + confusion_matrix(y_test, y_predictions)[1][1]
entry_tax = (entry_games / test_total_matches) * 100
hit_rate = (confusion_matrix(y_test, y_predictions)[1][1] / entry_games) * 100

print(f"Total Matches: {test_total_matches}")
print(f"Entry Games: {entry_games}")
print(f"Entry Tax: {entry_tax.round(2)}%")
print(f"Hit Rate: {hit_rate.round(2)}%")

Games that was predicted as been 0 and was 0: 163
Games that was predicted as been 0 and was 1: 125
Games that was predicted as been 1 and was 0: 110
Games that was predicted as been 1 and was 1: 230
Total Matches: 628
Entry Games: 340
Entry Tax: 54.14%
Hit Rate: 67.65%


<h5>Prevision / Probabilities of classification</h5></h5>

In [18]:
knn = KNeighborsClassifier(n_neighbors=3, metric="minkowski", p=2)
knn.fit(X, y)
predict_probability = knn.predict_proba(X)
df["predictions"] = y_predictions = knn.predict(X)
df["probability_0"] = predict_probability[:, 0].round(2)
df["probability_1"] = predict_probability[:, 1].round(2)
df.tail()

,league,date,home,away,home_odds,draw_odds,away_odds,PSH,PSD,PSA,home_goals_ft,away_goals_ft,lay_home,predictions,probability_0,probability_1
2506,Premier League,2023-02-18,Nott'm Forest,Man City,8.50,5.50,1.30,10.30,5.67,1.33,1,1,1.0,1.0,0.00,1.00
2507,Premier League,2023-02-18,Wolves,Bournemouth,1.67,3.60,6.00,1.69,3.86,5.78,0,1,1.0,1.0,0.00,1.00
2508,Premier League,2023-02-18,Newcastle,Liverpool,2.50,3.50,2.70,2.57,3.58,2.82,0,2,1.0,1.0,0.33,0.67
2509,Premier League,2023-02-19,Man United,Leicester,1.62,4.33,5.25,1.65,4.34,5.37,3,0,0.0,0.0,1.00,0.00
2510,Premier League,2023-02-19,Tottenham,West Ham,1.75,3.60,4.75,1.87,3.76,4.47,2,0,0.0,0.0,1.00,0.00
